<a href="https://colab.research.google.com/github/aadumbuya/Formative_2_Deep_Q_Learning/blob/main/Play_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install stable-baselines3[extra] gymnasium[atari] ale-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [5]:
"""
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback
import torch
import os
import ale_py
"""

'\nimport gymnasium as gym\nfrom stable_baselines3 import DQN\nfrom stable_baselines3.common.evaluation import evaluate_policy\nfrom stable_baselines3.common.monitor import Monitor\nfrom stable_baselines3.common.callbacks import CheckpointCallback\nimport torch\nimport os\nimport ale_py\n'

In [14]:
!apt-get install -y xvfb python3-opengl ffmpeg
#!pip install PyOpenGL
!pip install pyvirtualdisplay


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  libgle3 python3-numpy
The following NEW packages will be installed:
  freeglut3 libfontenc1 libglu1-mesa libxfont2 libxkbfile1 python3-opengl x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 29 not upgraded.
Need to get 8,638 kB of archives.
After this operation, 20.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 freeglut3 amd64 2.8.1-6 [74.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main

In [18]:
import gymnasium as gym
import time
import imageio
from IPython.display import HTML
from base64 import b64encode
from stable_baselines3 import DQN
from pyvirtualdisplay import Display

# Start a virtual display for headless environments (e.g., Colab)
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video(frames, fps=30):
    video_file = "gameplay.mp4"
    # Use imageio.get_writer with macro_block_size=1 to avoid automatic resizing
    writer = imageio.get_writer(video_file, fps=fps, macro_block_size=1)
    for frame in frames:
        writer.append_data(frame)
    writer.close()

    with open(video_file, "rb") as f:
        mp4 = f.read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
        <video width="600" controls>
            <source src="{data_url}" type="video/mp4">
        </video>
    """)

def main():
    # Use 'rgb_array' mode to capture frames for video recording
    env = gym.make("ALE/Pong-v5", render_mode="rgb_array")

    # Load the trained DQN model
    model = DQN.load("./models/dqn_model.zip")

    frames = []
    obs, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Capture the current frame
        frame = env.render()  # returns a numpy array image frame
        frames.append(frame)

        # Select action using a greedy (deterministic) policy
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        total_reward += reward

        # Slow down the loop for demonstration purposes
        time.sleep(0.02)

        if done or truncated:
            break

    env.close()
    print(f"Total Reward: {total_reward}")
    return frames

from IPython.display import display

if __name__ == "__main__":
    frames = main()
    # Capture the HTML output
    video_html = show_video(frames)
    # Explicitly display it
    display(video_html)


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/buffers.py:242: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 201.62GB > 11.24GB
  warnings.warn(


Total Reward: -21.0
